In [1]:
import os
os.chdir("../")
os.getcwd()

'c:\\Users\\hp\\Documents\\ML_Intern'

In [2]:
from Concrete_CS.constant import *
from Concrete_CS.utils import read_yaml, create_directories

In [13]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_transformation_path: Path
    train_dataset_features_path: Path
    train_dataset_target_path: Path
    test_dataset_features_path: Path
    test_dataset_target_path: Path
    test_size: float
    random_state: int

In [25]:
class ConfigurationManager:
    def __init__(self,
                config_filepath= CONFIG_FILE_PATH,
                params_filepath= PARAMS_FILE_PATH):
                self.config= read_yaml(config_filepath)
                self.params= read_yaml(params_filepath)
                create_directories([self.config.artifacts_root])
    def get_data_transformation_config(self)-> DataTransformationConfig:
        config= self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config= DataTransformationConfig(
                                root_dir= Path(config.root_dir),
                                data_transformation_path= Path(config.data_transformation_path),
                                train_dataset_features_path= Path(config.train_dataset_features_path),
                                train_dataset_target_path= Path(config.train_dataset_target_path),
                                test_dataset_features_path= Path(config.test_dataset_features_path),
                                test_dataset_target_path= Path(config.test_dataset_target_path),
                                test_size= float(self.params.train_test_split.params.test_size),
                                random_state= int(self.params.train_test_split.params.random_state)
                                )
        return data_transformation_config

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config= config
    def train_test_data(self):
        data= pd.read_csv(self.config.data_transformation_path)
        training_data= data.iloc[:,1:-1]
        testing_data= data.iloc[:,-1]
        X_train, X_test, y_train, y_test= train_test_split(training_data,
                                                           testing_data,
                                                           test_size=self.config.test_size,
                                                           random_state= self.config.random_state)
        self.save_data(X_train, self.config.train_dataset_features_path)
        self.save_data(y_train, self.config.train_dataset_target_path)
        self.save_data(X_test, self.config.test_dataset_features_path)
        self.save_data(y_test, self.config.test_dataset_target_path)
    def save_data(self,save_file,save_file_path):
        data= pd.DataFrame(save_file)
        data.to_csv(save_file_path)
        new_data= pd.read_csv(save_file_path)
        return new_data

In [28]:
config= ConfigurationManager()
data_transformation_config= config.get_data_transformation_config()
data_transformation= DataTransformation(config= data_transformation_config)
data_transformation.train_test_data()